In [58]:
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest
import botocore
import boto3
import json
from typing import Any, Dict, Optional, Union
import requests
import os
from utils import create_connection


def signed_request(
    credentials: botocore.credentials.ReadOnlyCredentials,
    method: str,
    url: str,
    service_name: str,
    region: str,
    data: Optional[Union[str, Dict[str, Any]]] = None,
    params: Optional[Dict[str, str]] = None,
    headers: Optional[Dict[str, str]] = None,
) -> requests.Response:
    """
    Sign a request using SigV4 and return its response.

    :param credentials: The boto3 session credentials.
    :param method: HTTP method (GET, POST, etc.).
    :param url: The endpoint URL.
    :param service_name: The AWS service name (e.g., execute-api for API Gateway, lambda for Lambda).
    :param region: AWS region (e.g., us-east-1).
    :param data: Request payload (optional).
    :param params: Query parameters (optional).
    :param headers: HTTP headers (optional).
    """
    headers = headers or {}

    if isinstance(data, dict):
        headers["Content-Type"] = "application/json"
        data = json.dumps(data)
        headers["Content-Length"] = str(len(data.encode("utf-8")))

    request = AWSRequest(
        method=method, url=url, data=data, params=params, headers=headers
    )
    SigV4Auth(credentials, service_name, region).add_auth(request)

    return requests.request(
        method=method,
        url=url,
        headers=dict(request.headers),
        data=data,
        allow_redirects=False,
    )


# Store the credentials for the user we created above.
credentials = {
    "AWS_ACCESS_KEY_ID": "...",
    "AWS_SECRET_ACCESS_KEY": "...",
}

In [59]:
# Get similar queries to the one you're asking from the vector db.
# Create an active session based on the credentials
session = boto3.Session(
    aws_access_key_id=credentials["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=credentials["AWS_SECRET_ACCESS_KEY"],
    region_name="us-west-2",
)
creds = session.get_credentials().get_frozen_credentials()

response = signed_request(
    creds,
    "GET",
    "https://dwaudf3qsmncsapnetmoikbmai0daihc.lambda-url.us-west-2.on.aws/find/?query=average%20revenue",
    service_name="lambda",
    region="us-west-2",
)
print(response.text)

[["average_annual_revenue_by_closing","\n    -- The average annual revenue of institutions with a high chance of closing?\n    SELECT \n        AVG(\"Annual Revenue\") AS average_annual_revenue\n    FROM \n        all_companies\n    -- Filter the rows to include only those with a high likelihood of closing.\n    WHERE \n        \"Likelihood to close\" >= {likelihood_threshold};\n    ",["likelihood_threshold"],["number"],0.12468554075783977],["total_revenue_by_country","\n    -- This query calculates total revenue by country/region.\n    SELECT \n        \"Country/Region\", \n        SUM(\"Annual Revenue\") AS total_revenue\n    FROM \n        all_companies\n    GROUP BY \n        \"Country/Region\"\n    ORDER BY \n        total_revenue {order};\n    ",["order"],["string"],0.1478357226946383],["top_10_companies_by_revenue","\n    -- This query lists the top 10 companies based on their annual revenue.\n    SELECT \n        \"Company name\", \n        \"Annual Revenue\"\n    FROM \n      

In [60]:
# Make a query call where we ask something in human language.
# Create an active session based on the credentials
session = boto3.Session(
    aws_access_key_id=credentials["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=credentials["AWS_SECRET_ACCESS_KEY"],
    region_name="us-west-2",
)
creds = session.get_credentials().get_frozen_credentials()

response = signed_request(
    creds,
    "POST",
    "https://dwaudf3qsmncsapnetmoikbmai0daihc.lambda-url.us-west-2.on.aws/query",
    service_name="lambda",
    region="us-west-2",
    data={"query": "how many deals have we closed?"},
)
print(response.text)

[[1939]]


In [61]:
# Read the user query that was saved to the DB.
# Assuming you have already created a connection
CREDENTIALS = {
    "password": os.getenv("DB_PASSWORD", "..."),
    "host": os.getenv(
        "DB_HOST",
        "mainstackrdsstackb4b88b4d-postgresvectordb82399e33-y5fzuij1hel2.cbas6w2cunpd.us-west-2.rds.amazonaws.com",
    ),
    "port": os.getenv("DB_PORT", "1053"),
    "ssl_path": "/Users/tetracycline/repos/rag-tutorial/us-west-2-bundle.pem",
}
conn = create_connection(**CREDENTIALS)
cur = conn.cursor()
# Execute the command
cur.execute("SELECT * FROM user_queries;")

# Commit the transaction and close the connection
out = cur.fetchall()
cur.close()
conn.close()

The error 'connection to server at "mainstackrdsstackb4b88b4d-postgresvectordb82399e33-y5fzuij1hel2.cbas6w2cunpd.us-west-2.rds.amazonaws.com" (127.0.0.1), port 1053 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
' occurred


AttributeError: 'NoneType' object has no attribute 'cursor'

In [47]:
out

[(1,
  'What is the average revenue of high-closing institutions?',
  'WITH high_chance_deals AS (\n    SELECT \n        d."Record ID" AS deal_id,\n        d."Deal probability" AS probability,\n        dc."Revenue" AS annual_revenue\n    FROM \n        public.all_deals d\n    JOIN \n        public.all_contacts c ON d."Record ID" = c."Record ID"\n    JOIN \n        (\n            SELECT \n                "Record ID",\n                SUM("Event Revenue") AS "Revenue"\n            FROM \n                public.all_contacts\n            GROUP BY \n                "Record ID"\n        ) dc ON c."Record ID" = dc."Record ID"\n    WHERE \n        d."Deal probability" > 0.8  -- Adjust this threshold based on the definition of "high chance" of closing\n)\nSELECT \n    AVG(hcd.annual_revenue) AS average_annual_revenue\nFROM \n    high_chance_deals hcd;',
  '[{"role": "system", "content": ""}, {"role": "user", "content": "\\nI ran this query on my database:\\n\\n```\\n\\nSELECT \\n    table_schem

In [57]:
# Make a query call where we ask something in human language that we already have in our table
# Create an active session based on the credentials
session = boto3.Session(
    aws_access_key_id=credentials["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=credentials["AWS_SECRET_ACCESS_KEY"],
    region_name="us-west-2",
)
creds = session.get_credentials().get_frozen_credentials()

response = signed_request(
    creds,
    "POST",
    "https://dwaudf3qsmncsapnetmoikbmai0daihc.lambda-url.us-west-2.on.aws/query",
    service_name="lambda",
    region="us-west-2",
    data={
        "query": "calculate the average annual revenue of institutions with a high chance of closing?"
    },
)
print(response.text)

Internal Server Error
